In [1]:
 !pip install torch torchvision matplotlib --quiet

import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Reproducibility
torch.manual_seed(42)
random.seed(42)


device = torch.device("cpu")

print("Using device:", device)


IndentationError: unexpected indent (2064970627.py, line 1)